In [ ]:
import os
import glob
from bs4 import BeautifulSoup
import openai
import markdownify

# Replace this with your OpenAI API key
openai.api_key = 'your_openai_api_key'

def extract_post_content(html_file):
    with open(html_file, 'r', encoding='utf-8') as file:
        content = file.read()
        soup = BeautifulSoup(content, 'html.parser')
        post_section = soup.find(id='post')
        if post_section:
            return post_section.get_text()
    return None

def send_to_openai_api(text, prompt):
    response = openai.Completion.create(
        engine="davinci-codex",
        prompt=f"{text}\n{prompt}",
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.8,
    )
    return response.choices[0].text.strip()

def save_markdown_file(content, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(content)

input_folder = 'path/to/your/html_files'
output_folder = 'path/to/your/markdown_files'
prompt = 'Your specified prompt here'

html_files = glob.glob(os.path.join(input_folder, '*.html'))

for html_file in html_files:
    post_content = extract_post_content(html_file)
    if post_content:
        api_response = send_to_openai_api(post_content, prompt)
        md_content = markdownify.markdownify(api_response)
        output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(html_file))[0] + '.md')
        save_markdown_file(md_content, output_file)
